<a href="https://colab.research.google.com/github/LeonardoVieiraGuimaraes/MiniCursoPalestra/blob/main/redeNeuralArtificial/ObjetoQueda/CorpoQueda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explicação: preparação do ambiente (Colab vs local)

Esta célula detecta o ambiente e garante dependências mínimas antes de executar o notebook:

- Detecta se está no Google Colab.  
  - Se sim: tenta montar o Google Drive (para salvar modelos) e instala apenas pacotes ausentes, evitando reinstalar pacotes já presentes no runtime do Colab.  
  - Se não: cria/usa uma virtualenv em `.venv` e instala os pacotes no pip do venv.
- Instalação é idempotente: apenas pacotes faltantes serão instalados; no ambiente local, ative o `.venv` manualmente antes de continuar.
- Execute esta célula primeiro. Em Colab, autorize o mount quando solicitado; no local, ative o venv (Windows):  
  - `.venv\Scripts\activate` (cmd) ou `.venv\Scripts\Activate.ps1` (PowerShell)
- Observações:
  - Instalar TensorFlow pode levar alguns minutos e exigir reinício do kernel; siga mensagens exibidas.
  - Se tiver problemas de permissão no Windows, execute o terminal como administrador.

In [5]:
# ...existing code...
import sys
import subprocess
import importlib

# pacotes requeridos: chave = nome pip, valor = nome do módulo para import
REQUIRED = {
    "numpy": "numpy",
    "pandas": "pandas",
    "scikit-learn": "sklearn",
    "matplotlib": "matplotlib",
    "joblib": "joblib",
    "tensorflow": "tensorflow"
}

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])

print("Instalando dependências ausentes (sem montar Drive)...")
for pkg, mod in REQUIRED.items():
    try:
        importlib.import_module(mod)
        print(f"  {mod}: ok")
    except Exception:
        print(f"  {mod}: não encontrado — instalando {pkg} ...")
        install(pkg)

def versoes():
    out = {}
    for pkg, mod in REQUIRED.items():
        try:
            m = importlib.import_module(mod)
            out[mod] = getattr(m, "__version__", "ok")
        except Exception:
            out[mod] = None
    return out

print("Versões (None = não disponível):", versoes())
# ...existing code...

Instalando dependências ausentes (sem montar Drive)...
  numpy: ok
  pandas: ok
  sklearn: ok
  matplotlib: ok
  joblib: ok
  tensorflow: ok
Versões (None = não disponível): {'numpy': '2.3.3', 'pandas': '2.3.3', 'sklearn': '1.7.2', 'matplotlib': '3.10.7', 'joblib': '1.5.2', 'tensorflow': '2.20.0'}


# 1 — Comentário e explicação (arrasto linear)

Descrição curta
- Essa célula calcula a velocidade v(t) e a posição y(t) de um corpo em queda sujeito a arrasto linear (força de arrasto ∝ v), usando a solução analítica da EDO.

Variáveis / unidades
- s_0 = posição inicial (m) — definida, mas não somada em y_analitica no código atual (ver observação).
- m = massa (kg)
- k = coeficiente de arrasto linear (N·s/m ou kg/s dependendo da formulação)
- t = tempo (s)
- g = aceleração da gravidade (m/s²)

Equações e significado
- Equação diferencial para a velocidade (modelo): dv/dt = g - (k/m)·v, com v(0)=0.
- Solução analítica da velocidade:
  v(t) = (m·g / k) · (1 - exp(-k·t / m))
  - Interpretação: v(t) parte de 0 e tende assintoticamente à velocidade terminal v∞ = m·g / k.
- Posição (integração de v(t), assumindo s(0)=s_0):
  y(t) = s_0 + (m·g / k)·t - (m²·g / k²)·(1 - exp(-k·t / m))
  - No código atual s_0 é declarado mas não adicionado; a expressão usada corresponde a s_0 = 0.
  - Comportamento: para t → 0, y ≈ s_0 + 0; para t grande, y ≈ s_0 + v∞·t - (m²·g / k²).

Detalhes de implementação / observações
- O código usa np.exp(...) — certifique-se de importar numpy como `import numpy as np` antes de executar.
- Evitar k = 0 (divisão por zero). Se quiser modelar sem arrasto use a solução sem termos em k.
- Para incluir a posição inicial corretamente, some `s_0` em `y_analitica`.
- Uso prático: útil para gerar datasets sintéticos, validar integradores numéricos (RK4) ou comparar previsões de modelos de ML com soluções físicas.

Exemplo de correção mínima (para incluir s_0):
```python
# adicionar s_0 à posição analítica
y_analitica = s_0 + (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))
```

In [6]:
import numpy as np
# Parâmetros de exemplo para demonstração
s_0 = 0.0
m = 5.0  # kg
k = 0.5  # coeficiente de arrasto (modelo linear usado aqui)
t = 5.0  # segundos
g = 9.81  # aceleração da gravidade (m/s^2)

# Fórmulas analíticas (modelo com arrasto linear)
v_analitica = (m * g / k) * (1 - np.exp(-k * t / m))
y_analitica = (m * g / k) * t - (m**2 * g / k**2) * (1 - np.exp(-k * t / m))

print(f'Velocidade analítica: {v_analitica:.4f} m/s')
print(f'Posição analítica: {y_analitica:.4f} m')

Velocidade analítica: 38.5993 m/s
Posição analítica: 104.5066 m


# 2 — Geração do dataset (arrasto linear)

## Propósito
Gerar um dataset sintético com entradas (massa, constante de arrasto, tempo) e saídas (velocidade, posição) usando a solução analítica do modelo com arrasto linear, e salvar em CSV.

## Pré-requisitos
- Importar: `import numpy as np`, `import pandas as pd`, `import os`.
- (Opcional) Variável `IN_COLAB` definida na Célula 0 para decidir salvar no Google Drive.

## Equações usadas
- dv/dt = g − (k/m)·v, v(0)=0  
  v(t) = (m·g / k) · (1 − exp(−k·t / m))
- y(t) = (m·g / k)·t − (m²·g / k²)·(1 − exp(−k·t / m))  
  (essa expressão assume posição inicial s0 = 0; some s0 se necessário)

## Fluxo da célula
1. Define g = 9.81 e semente aleatória `np.random.seed(42)` (reprodutibilidade).  
2. Gera `n_samples` amostras aleatórias para massa (1–10 kg), k (0.1–1.0) e tempo (0–10 s).  
3. Calcula v(t) e y(t) para cada amostra (implementado com loop).  
4. Monta um DataFrame com colunas: `massa`, `constante_de_arrasto`, `tempo`, `velocidade`, `posicao`.  
5. Decide caminho de salvamento:
   - Se `IN_COLAB == True` (e Drive já montado), salva em `/content/drive/MyDrive/...`.
   - Caso contrário salva em `./data/queda_objeto.csv`.
6. Grava o CSV e exibe caminho salvo.

## Observações importantes
- Salvar em `./data/...` no Colab não grava no Google Drive. Para persistir no Drive é necessário escrever dentro de `/content/drive/...` após `drive.mount('/content/drive')`.
- Intervalo de `k` evita divisão por zero; se alterar o domínio garanta `k != 0`.
- Loop é funcional, mas versão vetorizada com NumPy é muito mais rápida para 1000+ amostras.
- Se quiser posição inicial não nula, inclua `s0` em y(t).
- Garanta `import numpy as np` e `import os` no escopo antes de executar.

In [7]:
# ...existing code...
import numpy as np
import pandas as pd
import os

# Célula 2 — gerar dataset (vetorizado)
g = 9.81
np.random.seed(42)
n_samples = 1000

massas = np.random.uniform(1.0, 10.0, n_samples)
k_arrastos = np.random.uniform(0.1, 1.0, n_samples)
tempos = np.random.uniform(0.0, 10.0, n_samples)

# cálculo vetorizado (sem loop)
velocidades = (massas * g / k_arrastos) * (1 - np.exp(-k_arrastos * tempos / massas))
posicoes = (massas * g / k_arrastos) * tempos - (massas**2 * g / k_arrastos**2) * (1 - np.exp(-k_arrastos * tempos / massas))

df = pd.DataFrame({
    'massa': massas,
    'constante_de_arrasto': k_arrastos,
    'tempo': tempos,
    'velocidade': velocidades,
    'posicao': posicoes
})

# decidir local de salvamento — sempre local (não salvar no Drive)
save_dir = 'data'
os.makedirs(save_dir, exist_ok=True)
csv_path = os.path.join(save_dir, 'queda_objeto.csv')
df.to_csv(csv_path, index=False)
print('Dataset salvo em', csv_path)


Dataset salvo em data\queda_objeto.csv


# 3 — pré‑processamento, split e salvamento dos scalers

## Propósito
Carregar o CSV gerado, separar entradas (X) e saídas (y), dividir em treino/teste, padronizar features e targets com StandardScaler e salvar os scalers para inferência posterior.

## Pré‑requisitos
- Arquivo `data/queda_objeto.csv` presente com colunas: `massa`, `constante_de_arrasto`, `tempo`, `velocidade`, `posicao`.  
- Bibliotecas: pandas, numpy, scikit‑learn, joblib (tensorflow não é obrigatório nesta célula).

## Passos (resumo)
1. Verifica existência do CSV e aborta se ausente.  
2. Lê o CSV para DataFrame.  
3. Separa X = ['massa', 'constante_de_arrasto', 'tempo'] e y = ['velocidade', 'posicao'].  
4. Divide em treino/teste (test_size=0.2, random_state=42).  
5. Cria StandardScaler para X e y; ajusta (fit) apenas em X_train/y_train e transforma X_test/y_test.  
6. Cria pasta `model/` e salva `scaler_X.pkl` e `scaler_y.pkl` com joblib.  
7. Imprime confirmação.


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Carregar os dados do CSV (caminho atualizado)
csv_path = 'data/queda_objeto.csv'
if not os.path.exists(csv_path):
    raise SystemExit(f'Arquivo de dados não encontrado: {csv_path} - execute a célula de geração de dados primeiro')
df = pd.read_csv(csv_path)

# Separar as entradas (X) e as saídas (y)
X = df[['massa', 'constante_de_arrasto', 'tempo']]
y = df[['velocidade', 'posicao']]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar os dados
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# salvar scalers localmente (não usar Drive)
save_model_dir = 'model'
os.makedirs(save_model_dir, exist_ok=True)

import joblib
joblib.dump(scaler_X, os.path.join(save_model_dir, 'scaler_X.pkl'))
joblib.dump(scaler_y, os.path.join(save_model_dir, 'scaler_y.pkl'))
print(f'Scalers salvos em {save_model_dir}')

Scalers salvos em model


# 4 — Construção e treino do modelo Keras

## Propósito
Definir a arquitetura da rede neural, compilar com otimizador e função de perda, treinar com callbacks (salvar melhor modelo e EarlyStopping) e salvar o modelo final.  

## Entradas esperadas
- X_train, y_train: dados de treino já pré-processados (StandardScaler aplicado).
- save_model_dir: caminho onde salvar modelos e checkpoints (a célula a seguir detecta Colab e ajusta automaticamente).

## Passos
1. Construir modelo Sequential com camadas Dense (duas camadas ocultas ReLU + saída linear com 2 neurônios).  
2. Compilar com optimizer Adam, loss MSE e métrica MAE.  
3. Definir callbacks:
   - ModelCheckpoint para salvar o melhor modelo (monitorar val_loss).
   - EarlyStopping para parar cedo e restaurar melhores pesos.  
4. Treinar com validation_split (ou use um val set separado).  
5. Salvar o modelo final no diretório definido.  

## Observações
- Em Colab, monte o Drive (`drive.mount('/content/drive')`) antes para persistir modelos.  
- Ajuste epochs, batch_size e arquitetura conforme necessidade.  
- Para deploy/inferência, carregue o scaler_X/scaler_y salvos e o modelo salvo.

In [ ]:
from tensorflow.keras import Input

# Construir o modelo (corrigido: usar Input em vez de input_dim na Dense)
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='linear')  # Saída de 2 dimensões: velocidade e posição
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# Treinar o modelo com callbacks (salvar melhor modelo)
checkpoint_cb = ModelCheckpoint('model/queda_objeto_best.keras', save_best_only=True, monitor='val_loss')
earlystop_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[checkpoint_cb, earlystop_cb])

# salvar o modelo final
model.save('model/queda_objeto.keras')
print('Modelo salvo em model/queda_objeto.keras')

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,546 (17.76 KB)

 Trainable params: 4,546 (17.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.7983 - mae: 0.7186 - val_loss: 0.4500 - val_mae: 0.5522
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2447 - mae: 0.3728 - val_loss: 0.0983 - val_mae: 0.2185
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0684 - mae: 0.1912 - val_loss: 0.0557 - val_mae: 0.1794
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0366 - mae: 0.1480 - val_loss: 0.0362 - val_mae: 0.1388
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0250 - mae: 0.1205 - val_loss: 0.0270 - val_mae: 0.1198
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0188 - mae: 0.1033 - val_loss: 0.0220 - val_mae: 0.1044
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0147 - mae: 0.0922 - val_loss: 0.0179 - val_mae: 0.0935
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0125 - mae: 0.0830 - val_loss: 0.0154 - val_mae: 0.0832
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.010

# Avaliação do modelo (explicação)

Objetivo
- Avaliar a performance do modelo treinado no conjunto de teste e exibir o erro absoluto médio (MAE).

O que a célula faz (passo a passo)
1. Define `best_path = 'model/queda_objeto_best.keras'` — caminho do checkpoint com o melhor modelo (salvo pelo `ModelCheckpoint` durante o treino).  
2. Se o arquivo em `best_path` existir, carrega esse modelo com `tensorflow.keras.models.load_model()` e sobrescreve a variável `model`. Isso garante que a avaliação use os pesos com melhor `val_loss`.  
3. Chama `model.evaluate(X_test, y_test)`:
   - Retorna uma tupla com `(loss, mae)` porque o modelo foi compilado com `loss='mse'` e `metrics=['mae']`.
   - `loss` corresponde ao MSE no conjunto de teste; `mae` é o Mean Absolute Error (impresso em seguida).  
4. Imprime o MAE formatado.

Observações / cuidados
- Garanta que `X_test` e `y_test` estejam definidos no escopo (vêm da célula de pré‑processamento). Caso contrário a avaliação falhará.  
- Se `best_path` não existir e a variável `model` não estiver definida (por exemplo, se você não executou a célula de treino nesta sessão), ocorrerá um erro. Alternativa segura: tentar carregar `model/queda_objeto.keras` ou instruir a executar a célula de treino primeiro.  
- `model.evaluate` já faz a normalização/inferência de lote internamente — certifique‑se de que `X_test`/`y_test` estejam pré‑processados da mesma forma usada no treino (scalers aplicados).

Dica rápida (prática)
- Para maior robustez, verifique ambos os arquivos (`queda_objeto_best.keras` e `queda_objeto.keras`) e forneça mensagens claras ao usuário sobre o que foi carregado ou se é necessário treinar o modelo primeiro.

In [ ]:
# Avaliar o modelo (carregar melhor modelo salvo se existir)
import os
best_path = 'model/queda_objeto_best.keras'
if os.path.exists(best_path):
    from tensorflow.keras.models import load_model
    model = load_model(best_path)
    print('Carregado melhor modelo salvo em', best_path)

loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae:.6f}')

# Modelo final já salvo na etapa de treino como model/queda_objeto.keras

# de inferência — explicação

Objetivo
- Carregar o modelo treinado e os scalers, preparar uma entrada, prever (rede) e converter a saída padronizada de volta às unidades originais. Também compara com a solução analítica.

Passo a passo (resumido)
1. Verificação e carregamento  
   - Verifica existência de `model/queda_objeto.keras`; se ausente interrompe com mensagem.  
   - Carrega o modelo com `load_model(model_path)` e os scalers com `joblib.load(...)`.

2. Preparar entrada de exemplo  
   - Define m_val, k_val, t_val como escala 1×3 (massa, constante de arrasto, tempo).  
   - Se o `scaler_X` foi treinado a partir de um DataFrame, pode ter `feature_names_in_`; nesse caso cria um DataFrame com essas colunas para evitar desalinhamento de features.  
   - Senão, usa um numpy array 2D (shape (1,3)).  
   - Resultado: `entrada_pad` (features padronizadas).

3. Previsão e pós‑processamento  
   - Chama `model.predict(entrada_pad)` → retorna `saida_pad` (saída padronizada, shape (1,2)).  
   - Converte de volta para escala original com `scaler_y.inverse_transform(saida_pad)` → `saida` (velocidade, posição).

4. Extração e impressão  
   - Extrai `v_pred, y_pred = saida[0]` e imprime.  
   - Calcula solução analítica com as mesmas entradas e imprime para comparação.

Pontos importantes / cuidados
- Garantir que `scaler_X` e `scaler_y` correspondam exatamente às transformações usadas no treino (mesmas colunas/ordem).  
- `model.predict` espera entrada 2D; não passar 1D.  
- Se o scaler foi criado com DataFrame e as colunas estiverem em ordem diferente, usar `feature_names_in_` evita erro.  
- Se o modelo foi treinado em dados padronizados, nunca fornecer entradas não padronizadas diretamente ao modelo.  
- Conferir shapes: entrada (1,3) → saída (1,2).  
- As comparações com a solução analítica servem para validação simples, mas diferenças são esperadas (erro de modelagem/treino).

Sugestão rápida
- Para testar múltiplas entradas, empilhar linhas em `df_input` ou em um array (n_samples, 3) e usar o mesmo pipeline de transformação antes de chamar `model.predict`.

In [ ]:
# Célula de inferência — carregar modelo e scalers e testar uma entrada
import numpy as np
import os
from tensorflow.keras.models import load_model
import joblib

model_path = 'model/queda_objeto.keras'
if not os.path.exists(model_path):
    raise SystemExit('Modelo não encontrado: ' + model_path + ' - execute a célula de treino primeiro')
model = load_model(model_path)
scaler_X = joblib.load('model/scaler_X.pkl')
scaler_y = joblib.load('model/scaler_y.pkl')

# Exemplo de entrada e previsão
m_val = 5.0
k_val = 0.5
t_val = 5.0

# Preparar entrada preservando nomes de features quando o scaler os tem
try:
    import pandas as pd
    if hasattr(scaler_X, 'feature_names_in_'):
        cols = list(scaler_X.feature_names_in_)
        df_input = pd.DataFrame([[m_val, k_val, t_val]], columns=cols)
        entrada_pad = scaler_X.transform(df_input)
    else:
        entrada = np.array([[m_val, k_val, t_val]])
        entrada_pad = scaler_X.transform(entrada)
except Exception as e:
    # fallback simples se pandas não estiver disponível ou ocorrer erro
    entrada = np.array([[m_val, k_val, t_val]])
    entrada_pad = scaler_X.transform(entrada)

saida_pad = model.predict(entrada_pad)
# Para scaler_y, verificar se espera colunas nomeadas também
try:
    # scaler_y geralmente não tem feature_names_in_, mas mantemos o fluxo
    saida = scaler_y.inverse_transform(saida_pad)
except Exception:
    saida = scaler_y.inverse_transform(saida_pad)

v_pred, y_pred = saida[0]
print(f'Previsão da rede — velocidade: {v_pred:.4f} m/s, posição: {y_pred:.4f} m')

# comparar com solução analítica
v_analitica = (m_val * 9.81 / k_val) * (1 - np.exp(-k_val * t_val / m_val))
y_analitica = (m_val * 9.81 / k_val) * t_val - (m_val**2 * 9.81 / k_val**2) * (1 - np.exp(-k_val * t_val / m_val))
print(f'Analítica — velocidade: {v_analitica:.4f} m/s, posição: {y_analitica:.4f} m')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Previsão da rede — velocidade: 38.3446 m/s, posição: 103.2006 m
Analítica — velocidade: 38.5993 m/s, posição: 104.5066 m
